In [2]:
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pandas as pd
import re
# Charger les variables d'environnement
load_dotenv()

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
TIMESTAMP_FILE = 'timestamp_last_tweet.txt'

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def get_game_tags_and_check_ai(app_id):
    try:
        url = f"https://store.steampowered.com/app/{app_id}/"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Vérifier la présence de contenu généré par IA
            ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
            if ai_disclosure:
                print(f"Contenu AI détecté pour le jeu {app_id}")
                return None, None  # Le jeu utilise du contenu généré par IA
            
            # Récupérer les tags
            tag_elements = soup.find_all('a', class_='app_tag')
            tags = [tag.text.strip() for tag in tag_elements]
            return tags[:4], 'scraped'  # Retourne seulement les 4 premiers tags
    except Exception as e:
        print(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
    
    return None, None

In [3]:
get_game_details(3068400)

{'type': 'game',
 'name': 'VEGAS Pro 22 Steam Edition',
 'steam_appid': 3068400,
 'required_age': 0,
 'is_free': False,
 'detailed_description': 'The <strong>VEGAS Pro 22</strong> - Steam Edition has everything you need to edit your gaming videos, be it for your streaming highlights or exciting gameplays. Stay in the flow by editing video and audio on the unique all-in-one timeline and get a complete post-production kit for the finishing touches.<br><br>Whether you want to show off your own tactics, uncover Easter eggs or share valuable tips, the VEGAS Pro Steam Edition gives you the creative freedom to do so. For example, automatically mask individual elements, such as characters, and apply targeted effects to them. Set the highlights in scene with zooms, slow motion and fast motion. Or create your own transitions, effects and color looks to give your videos your own personal visual style.<h2 class="bb_tag">Highlights</h2><ul class="bb_ul"><li>Seamless editing of video and audio as we